## Kurze Beschreibung zu den Bearbeitungsschritten:


srtm:
1. Download der verschiedenen Kacheln über den jeweiligen Dienst
2. Mergen in QGIS und exportieren und speichern.
3. Spatial resolution an die des sat anpassen (Erst die sat Datei bearbeiten, da dies evtl. zu einer veränderung der sr führen kann!)

sat:
1. Mit GDAL gdalbuildvrt -separate rgb.vrt red.tif green.tif blue.tif
2. Mit GDAL gdalwarp -co "TILED=YES" -co BLOCKXSIZE=256 -co BLOCKYSIZE=256 -t_srs EPSG:xxxx rgb.vrt name_translate.tif   <-- add -srcnodata "0" to delete black edges!
3. In QGIS Laden und schauen ob es vernünftig dargestellt wird
4. Aus QGIS exportieren und darauf achten das Haken bei "wie dargestellt" gesetzt ist
5. Dieses skript drüber laufen lassen incl. des crop Teils. Anscheinend wird das Bild nur als .png vernünftig dargestellt


In [1]:
# Load the needed libraries

import numpy as np
import rasterio
import geopandas as gpd
from shapely.geometry import box
from rasterio.mask import mask
from rasterio.plot import show
from matplotlib import pyplot as plt
import progressbar
import os
import skimage
from skimage import io
from skimage.io import imsave
from PIL import Image
import os, os.path
import pathlib
from pathlib import Path
import progressbar
import matplotlib.image as mpimg

## Extraction of images as np arrays

In [2]:
# Read the file with rasterio
#data_dem = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/DEM_Aletsch_Winter.tif") #AletschAlpsWinter
#data_sat = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/SAT_Aletsch_Winter.tif") #AletschAlpsWinter

#data_dem = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/DEM_Bernese_Alps_summer.tif") #BerneseAlpsSummer
#data_sat = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/SAT_Bernese_Alps_summer.tif") #BerneseAlpsSummer

#data_dem = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/DEM_Alps_Summer_3.tif") #Alps3
#data_sat = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/SAT_Alps_Summer_3.tif") #Alps3

#data_dem = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/DEM_Alps_Summer.tif") #Alps1
#data_sat = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/SAT_Alps_Summer.tif") #Alps1

#data_dem = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/DEM_Alps_Summer_2.tif") #Alps2
#data_sat = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/SAT_Alps_Summer_2.tif") #Alps2



#data_dem = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/DEM_Allgaeu_Summer.tif") #AllgaeuSummer
#data_sat = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/SAT_Allgaeu_Summer.tif") #AllgaeuSummer

#fp_dem = os.path.join('/mnt', 'd', 'sciebo', 'BSc_Fehling_landscape_Gen', 'dem_sat','DEM_Bernese_Alps_winter.tif') #Bernese Winter
#fp_sat = os.path.join('/mnt', 'd', 'sciebo', 'BSc_Fehling_landscape_Gen', 'dem_sat','SAT_Bernese_Alps_Winter.tif') #Bernese Sinter

#fp_modis = os.path.join('/mnt','c','Users','robin','sciebo','EarthExplorer', 'Nepal', 'test_modis.tif') #modis image generated in qgis


#fp_sat = os.path.join('/mnt','c','Users','robin','sciebo','BSc_Fehling_Landscape_Gen','dem_sat','SAT_Aletsch_Winter.tif')
#fp_cop_dem = os.path.join('/mnt','c','Users','robin','sciebo','EarthExplorer','eu_dem_epsg_3857.tif')

#fp_cop_dem = r'/mnt/c/Users/robin/sciebo/BSc_Fehling_Landscape_Gen/01b_DEM_SAT_pairs/eu_wgs84.tif'

#fp_sat_sentinel2a = os.path.join('/mnt','d','sciebo','BSc_Fehling_Landscape_Gen','Training Datasets','Nepal', 'HLSSentinel-2','1','sat','test_dtype','translated_epsg_3857.tif')
#fp_dem_sentinel2a = os.path.join('/mnt','d','sciebo','BSc_Fehling_Landscape_Gen','Training Datasets','Nepal', 'HLSSentinel-2','1','srtm','merged_dem_3857.tif')

#fp_sat_sentinel2a = os.path.join('/mnt','d','BSc_Fehling_Landscape_Gen','Training Datasets','Nepal','S2B','GRANULE','L2A','IMG_DATA','R10m','warped_exported.tif')
#fp_dem_sentinel2a = os.path.join('/mnt','d','BSc_Fehling_Landscape_Gen','Training Datasets','Nepal','S2B','srtm','merged_3857_res_increase.tif')


#fp_sat_sentinel2a = os.path.join('/mnt', 'd', 'BSc_Fehling_Landscape_Gen', 'Training Datasets', 'Himalaya', 'S2A', 'GRANULE', 'L2A', 'IMG_DATA', 'R10m', 'warped_exported.tif')
#fp_dem_sentinel2a = os.path.join('/mnt', 'd', 'BSc_Fehling_Landscape_Gen', 'Training Datasets', 'Himalaya', 'srtm', 'merged10m.tif')

#fp_sat_sentinel2a = os.path.join('/mnt', 'd', 'sciebo','BSc_Fehling_Landscape_Gen', 'Training Datasets', 'Oman', 'HLSSentinel','sat_merged_rgb_3857.tif')
#fp_dem_sentinel2a = os.path.join('/mnt', 'd', 'sciebo','BSc_Fehling_Landscape_Gen', 'Training Datasets', 'Oman', 'srtm','merged_3857.tif')

fp_sat = r'/mnt/veracrypt2/CGRE/raw/Himalaya10x10/warped_exported.tif'
fp_dem = r'/mnt/veracrypt2/CGRE/raw/Himalaya10x10/merged10m.tif'

data_sat = rasterio.open(fp_sat)
data_dem = rasterio.open(fp_dem)

In [3]:
def generate_random_bbox(data_sat, nx=5000, ny=5000):
    """Generate random bbox within domain size of images
    
    nx = int: cells in x-direction
    ny = int: cells in y-direction
    """
    # set bounds from sat image for now:
    global_min_x = data_sat.bounds[0]
    global_min_y = data_sat.bounds[1]
    global_max_x = data_sat.bounds[2]
    global_max_y = data_sat.bounds[3]

    # get random location
    min_x = np.random.randint(global_min_x, global_max_x-nx)
    max_x = min_x + nx
    min_y = np.random.randint(global_min_y, global_max_y-ny)
    max_y = min_y + ny

    
    bbox = box(min_x, min_y, max_x, max_y)

    geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0])
    
    return geo

def extract_images(data_sat, data_dem, geo):
    """Extract subimages from sat and dem
    """

    # Get the geometry coordinates
    coords = getFeatures(geo)

    # Clip the raster with the polygon
    out_img_sat, out_transform_sat = mask(data_sat, shapes=coords, crop=True)
    out_img_dem, out_transform_dtm = mask(data_dem, shapes=coords, crop=True)
    
    return out_img_sat, out_img_dem

def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [4]:
def reshape_and_reorder(out_img_sat, out_img_dem):
    # remove edges
    out_img_sat = out_img_sat[:,2:-2,2:-2] #orig: out_img_sat[:,2:-2,2:-2] 
    out_img_dem = out_img_dem[:,2:-2,2:-2]
    # reorder axes
    out_img_sat_reshape = np.moveaxis(out_img_sat, 0, -1)
    out_img_dem_reshape = np.moveaxis(out_img_dem, 0, -1)
    
    return out_img_sat_reshape, out_img_dem_reshape

In [5]:
#from https://stackoverflow.com/questions/52486237/how-to-turn-off-skimage-warnings
# else scikit image will display a warning for a lossy conversion of int16 to uint8 every time a picture is saved
import imageio.core.util

def ignore_warnings(*args, **kwargs):
    pass

imageio.core.util._precision_warn = ignore_warnings

In [6]:
# Get the training images. Cut out random images from the srtm and the satellite images.

fp_save_sat2 = r'/mnt/veracrypt2/CGRE/processed/processed_datasets/sat/out_img_%06d.png'      # path to sat directory
fp_save_dem2 = r'/mnt/veracrypt2/CGRE/processed/processed_datasets/npy_dem/out_img_%06d.npy'  # path to dem directory
n = 1000                                                                                      # Number of images one wants to extract

for i in progressbar.progressbar(range(n)):
    geo = generate_random_bbox(data_sat, nx=6200, ny=4700)                                    # nx and ny correspond to the sizes of the images
    out_img_sat, out_img_dem = extract_images(data_sat, data_dem, geo)
    out_img_sat_reshape, out_img_dem_reshape = reshape_and_reorder(out_img_sat, out_img_dem)
    
    imsave(fp_save_sat2%i, out_img_sat_reshape, check_contrast = False)                        # save the sat files as png
    
    np.save(fp_save_dem2 % i, out_img_dem_reshape)                                             #saving the dems into numpy array first because that 
                                                                                               # doesnt seem to be a problem and without it the images appear grey!
    im_2 = np.load(r'/mnt/veracrypt2/CGRE/processed/processed_datasets/npy_dem/out_img_%06d.npy' % i ) # Load the numpy array 
    img_DEM = Image.fromarray(im_2[:,:,0])                                                     # Convert array to image wit Pillow
    mpimg.imsave(r'/mnt/veracrypt2/CGRE/processed/processed_datasets/dem/out_img_%06d.png' % i, img_DEM, cmap='gist_earth' ) # Save images as png
    



100% (1000 of 1000) |####################| Elapsed Time: 0:03:31 Time:  0:03:31


# trying to reshape the image so all have the same resolution:

## Remember to clean the directories

In [8]:
#Reshape the images, so that all of the have the same size

# Here we get the min sizes of x and y from the image

list_ = r'/mnt/veracrypt2/CGRE/processed/processed_datasets/sat/' 
path, dirs, files = next(os.walk(list_))                                              # https://stackoverflow.com/questions/2632205/how-to-count-the-number-of-files-in-a-directory-using-python


length = len(files)
shapes_dem = []
shapes_sat = []
for i in progressbar.progressbar(range(length)):

   

    
    fp_dem = r'/mnt/veracrypt2/CGRE/processed/processed_datasets/dem/out_img_%06d.png' 
    fp_sat = r'/mnt/veracrypt2/CGRE/processed/processed_datasets/sat/out_img_%06d.png'     
    
    im_dem = io.imread(fp_dem %i)
    im_sat = io.imread(fp_sat %i)
    
    shapes_dem.append(im_dem.shape)
    shapes_sat.append(im_sat.shape)
    
x_dem = []
y_dem = []
x_sat = []
y_sat = []

for i in shapes_dem:
    x_dem.append(i[0])
    y_dem.append(i[1])
for i in shapes_sat:
    x_sat.append(i[0])
    y_sat.append(i[1])
    
maxx_dem = max(x_dem)
maxy_dem = max(y_dem)
minx_dem = min(x_dem)
miny_dem = min(y_dem)


maxx_sat = max(x_sat)
maxy_sat = max(y_sat)
minx_sat = min(x_sat)
miny_sat = min(y_sat)

if minx_dem > minx_sat:
    minx = minx_sat
else:
    minx = minx_dem
    
if miny_dem > miny_sat:
    miny = miny_sat
else:
    miny = miny_dem
    

100% (1000 of 1000) |####################| Elapsed Time: 0:00:12 Time:  0:00:12


In [10]:
# Now we reshape the images!

for i in progressbar.progressbar(range(length)):
    
    fp_dem = r'/mnt/veracrypt2/CGRE/processed/processed_datasets/dem/out_img_%06d.png' 
    fp_sat = r'/mnt/veracrypt2/CGRE/processed/processed_datasets/sat/out_img_%06d.png'
    
    im_dem = io.imread(fp_dem %i)
    im_sat = io.imread(fp_sat %i)
    
    im_dem_new = im_dem[:minx, :miny]
    im_sat_new = im_sat[:minx, :miny]
    
    imsave(fp_dem %i, im_dem_new, check_contrast = False)
    imsave(fp_sat %i, im_sat_new, check_contrast = False)

100% (1000 of 1000) |####################| Elapsed Time: 0:03:51 Time:  0:03:51


In [ ]:
# Cobine The dataset for the training with pix2pix
fp_combine = r'/mnt/veracrypt2/CGRE/pytorch-CycleGAN-and-pix2pix/datasets/combine_A_and_B.py'
fp_path_a = r'/mnt/veracrypt/CGRE/processed/training_datasets/himalaya10x10/A'
fp_path_b = r'/mnt/veracrypt/CGRE/processed/training_datasets/himalaya10x10/B'
fp_path_ab = r'/mnt/veracrypt/CGRE/processed/training_datasets/himalaya10x10/AB'

!python D:/CGRE/pytorch-CycleGAN-and-pix2pix/datasets/combine_A_and_B.py --fold_A {fp_path_a} --fold_B {fp_path_b} --fold_AB {fp_path_ab}